In [25]:
import csv
import json

json_dictionary = {}


def clean_for_url(input):
    return input.replace('\u2013','-')

def capitalise_End(input):
    # NOTE: expects a space already
    return input.replace('\u2013end', '\u2013End');

def change_dot_to_colon(input):
    return input.replace('.', ':')

def remove_asterisk(input):
    return input.replace('*', '')

def prettify(input):
    # comment
    input = capitalise_End(input)
    input = change_dot_to_colon(input)
    input = remove_asterisk(input)
    return input

with open('./input/all_readings.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        
        day_dictionary = {}
        day_dictionary['date_pretty'] = row['date_pretty']
        day_dictionary['day_description'] = row['day_description']
        
        day_dictionary['morning'] = {
            'readings' : [reading.strip() for reading in prettify(row['morning_prayers_together']).split(';')],
            'reading_link': clean_for_url(row['morning_prayers_link']),
            'audio_link': clean_for_url(row['morning_audio_link']),
        }
        
        day_dictionary['evening'] = {
            'readings' : [reading.strip() for reading in prettify(row['evening_prayers_together']).split(';')],
            'reading_link': clean_for_url(row['evening_prayers_link']),
            'audio_link': clean_for_url(row['evening_audio_link']),
        }
        
        json_dictionary[row['date']] = day_dictionary
        
print(json.dumps(json_dictionary, indent=4))

{
    "2020/12/01": {
        "date_pretty": "Tuesday, December 1, 2020",
        "day_description": "Week of 1st Sunday of Advent",
        "morning": {
            "readings": [
                "Psalm 80",
                "Isaiah 43:1\u201313",
                "Revelation 20"
            ],
            "reading_link": "Psalm+80%3B+Isaiah+43.1-13%3B+Revelation+20",
            "audio_link": "Ps.80,Isa.43,Rev.20"
        },
        "evening": {
            "readings": [
                "Psalm 74",
                "Isaiah 26:1\u201313",
                "Matthew 12:22\u201337"
            ],
            "reading_link": "Psalm+74%3B+Isaiah+26.1-13%3B+Matthew+12.22-37",
            "audio_link": "Ps.74,Isa.26,Matt.12"
        }
    },
    "2020/12/02": {
        "date_pretty": "Wednesday, December 2, 2020",
        "day_description": "Week of 1st Sunday of Advent",
        "morning": {
            "readings": [
                "Psalm 7",
                "Isaiah 43:14\u2013End",
           

In [26]:
with open('./output/data.js', 'w') as outputfile:
    outputfile.write('class Data {\r')
    outputfile.write('  data = ' + json.dumps(json_dictionary, indent=4) + ';\r')
    outputfile.write('  getData(){return this.data;}\r')
    outputfile.write('}')